In [3]:
import cv2
import numpy as np
import os

In [4]:
def extract_color_descriptor(img):
    # Compute the average red, green, and blue values as a basic color descriptor
    R = np.mean(img[:, :, 2])  # Note: OpenCV uses BGR format
    G = np.mean(img[:, :, 1])
    B = np.mean(img[:, :, 0])
    return np.array([R, G, B])

In [5]:
DATASET_FOLDER = "MSRC_ObjCategImageDatabase_v2"
OUT_FOLDER = "/Users/frankcholula/Workspace/school/cvprlab/cvprlab_python/lab3"

if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

In [ ]:
# Extract Image Descriptors from the Dataset